# Notebook for Cellpy batch processing

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import cellpy
from cellpy import prms
from cellpy import prmreader
from cellpy.utils import batch
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# prmreader.info()

In [ ]:
project = "my_project"
name = "experiment_one"
batch_col = "b01"

print(" INITIALISATION OF BATCH ".center(80, "="))
b = batch.init(name, project, batch_col=batch_col)

In [ ]:
# setting some prms
b.experiment.force_cellpy = True
b.experiment.export_raw = True
b.experiment.export_cycles = True
b.experiment.export_ica = True
b.experiment.all_in_memory = True  # store all data in memory, defaults to False

In [ ]:
b.create_info_df()

In [ ]:
b.experiment.journal.pages.head()

In [ ]:
b.create_folder_structure()

In [ ]:
b.load_and_save_raw()

In [ ]:
b.make_summaries()
print("---FINISHED---")

In [ ]:
b.plot_summaries()

In [ ]:
#b.experiment.status()

In [ ]:
b.summaries.head()

## Looking more in-depth

### OCV relaxation points
Picking out 5 points on each OCV relaxation curve (distributed by last, last/2, last/2/2, ..., first).

In [ ]:
from cellpy.utils.batch_tools.batch_analyzers import OCVRelaxationAnalyzer

analyzer = OCVRelaxationAnalyzer()
analyzer.assign(b.experiment)
analyzer.do()
ocv_df_list = analyzer.farms[0]
df = ocv_df_list[0]
df_up = df.loc[df.type == "ocvrlx_up", :]
df_down = df.loc[df.type == "ocvrlx_down", :]

print(df_up.head())
df_down = df_down.set_index("cycle")

fig, (ax1, ax2) = plt.subplots(2, sharex=True)
df_down.loc[:, ["point_00", "point_01", "point_02", "point_03", "point_04"]].plot(ax=ax1, legend=None)
df_down.loc[:, ["point_00", "point_01", "point_02", "point_03", "point_04"]].plot(ax=ax2)
ax1.set_ylim(0.0,1)
ax2.set_ylabel("voltage");

## Looking closer at some summary-plots

In [ ]:
# List all the possible summaries that exists
# Remark! its possible to set options so that more summaries are made (as long as they exist
#   in the CellpyData.dataset.dfsummary frame(s))
b.summary_columns

In [ ]:
discharge_capacity = b.summaries.discharge_capacity
charge_capacity = b.summaries.charge_capacity
coulombic_efficiency = b.summaries.coulombic_efficiency
ir_charge = b.summaries.ir_charge

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.plot(discharge_capacity)
ax1.set_ylabel("capacity ")
ax2.plot(ir_charge)
ax2.set_xlabel("cycle")
ax2.set_ylabel("resistance")

## Checking for more details for each cycle

#### 1. pick the CellpyData object for one of the cells

In [ ]:
# Lets check what cells we have
cell_labels = b.experiment.cell_names
cell_labels

In [ ]:
# OK, then I choose one of them
data = b.experiment.data["20181026_cen30_02_cc"]

#### 2. Get some voltage curves for some cycles and plot them
The method `get_cap` can be used to extract voltage curves.

In [ ]:
cap = data.get_cap(categorical_column=True)
cap.head()

In [ ]:
fig, ax = plt.subplots()
ax.plot(cap.capacity, cap.voltage)
ax.set_xlabel("capacity")
ax.set_ylabel("voltage")

In [ ]:
c,v = data.get_cap(method="forth")
fig, ax = plt.subplots()
ax.set_xlabel("capacity")
ax.set_ylabel("voltage")
ax.plot(c,v)

In [ ]:
c4,v4 = data.get_cap(cycle=4, method="forth-and-forth")
c10,v10 = data.get_cap(cycle=10, method="forth-and-forth")
fig, ax = plt.subplots()
ax.set_xlabel("capacity")
ax.set_ylabel("voltage")
ax.plot(c4,v4, "ro", label="cycle 4")
ax.plot(c10,v10, "bs", label="cycle 22")
ax.legend();

### Looking at some dqdv data

#### Get capacity cycles and make dqdv using the ica module

In [ ]:
from cellpy.utils import ica
v4, dqdv4 = ica.dqdv_cycle(
    data.get_cap(
        4, 
        categorical_column=True, 
        method = "forth-and-forth")
)

v10, dqdv10 = ica.dqdv_cycle(
    data.get_cap(
        10, 
        categorical_column=True, 
        method = "forth-and-forth")
)

plt.plot(v4,dqdv4, label="cycle 4")
plt.plot(v10, dqdv10, label="cycle 10")
plt.legend();

#### Put it in a for-loop for plotting many ica plots

In [ ]:
fig, ax = plt.subplots()
for cycle in data.get_cycle_numbers():
    d = data.get_cap(
            cycle, 
            categorical_column=True, 
            method = "forth-and-forth"
        )
    if not d.empty:
        v, dqdv = ica.dqdv_cycle(d)
        ax.plot(v, dqdv)
    else:
        print(f"cycle {cycle} seems to be missing or corrupted")
    